Ok, let's check capabilities of SkLearn befere using NNs

Which method to use? Of course ones that enlisted [here](https://github.com/lemikhovalex/DA_CV) (repo for ML classes at MIPT)
## Preprocess data
But before using data let's create features and labels

The following features are to be created, accoarding to papers about LSTM:
- $t_{w}$ - time passed between Sunday midnight and the event
- $t_{e}$ - time passed between the completion of the given event and the completion of the previous one
- $t_{t}$ - time passed between the start of the trace and the given event
- one-hot-encoded labels
- make smth like n-gramm, but for process and use it as features

### Test train split
Ok it seems to be rational not to take random transitions, but whole traces, cause `n-gramm` preprocessing will be conducted natively, if data is provided as a sequence

In [1]:
! pip install python-dateutil

You should consider upgrading via the 'c:\users\alemikhov\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import datetime
import random
from dateutil.parser import parse

In [3]:
df = pd.read_csv('datasets/bpi_12.csv')
df['timestamp'] = df['timestamp'].apply(lambda x: parse(x))

In [4]:
df

,timestamp,trace_id,activity
0,2011-10-01 11:45:13.917000+02:00,173688,W_Completeren aanvraag
1,2011-10-01 12:17:08.924000+02:00,173688,W_Nabellen offertes
2,2011-10-08 16:32:00.886000+02:00,173688,W_Nabellen offertes
3,2011-10-10 11:33:05.791000+02:00,173688,W_Nabellen offertes
4,2011-10-13 10:37:37.026000+02:00,173688,W_Valideren aanvraag
...,...,...,...
72408,2012-03-01 09:28:21.228000+01:00,214370,W_Afhandelen leads
72409,2012-03-01 09:27:52.388000+01:00,214373,W_Afhandelen leads
72410,2012-03-01 20:22:41.157000+01:00,214373,W_Completeren aanvraag
72411,2012-03-10 12:50:54.881000+01:00,214373,W_Nabellen offertes


Also dont want to deal with short traces - composing features as window will fail it(

In [5]:
traces = list(set(df['trace_id']))
print(f'There are {len(traces)} traces')
traces_len = {trace_id: df[df['trace_id'] == trace_id].shape[0] for trace_id in traces}
traces_time = {}
for trace_id in traces:
    _df = df[df['trace_id'] == trace_id]
    if _df.shape[0] > 7:
        traces_time[trace_id] = (_df['timestamp'].max() - _df['timestamp'].min()).total_seconds()
traces_time = pd.DataFrame({'trace_id': list(traces_time.keys()), 'tt': list(traces_time.values())})
traces_time.sort_values(by=['tt'], inplace=True)

There are 9658 traces


In [6]:
traces_time.head()

,trace_id,tt
1142,174421,1175.742
1084,174120,4601.686
1133,174376,5961.455
2700,184607,8422.561
2484,182194,8647.784


Let's calculate 4 equivalent quantiles 4 trace length distribution and sample 30% from each

In [7]:
n_qs = 4
sh = traces_time.shape[0]
qs = [0.]
qs.extend([traces_time[_i * sh // n_qs: (_i + 1) * sh // n_qs]['tt'].max() for _i in range(n_qs)])

In [8]:
test_ids = []
test_relative = 0.25
for _i in range(n_qs):
    _lb_cond = (traces_time['tt'] > qs[_i])
    _rb_cond = (traces_time['tt'] <= qs[_i+1])
    _cond = _lb_cond & _rb_cond
    _pop = traces_time[_cond]['trace_id'].values.tolist()
    test_ids.extend(random.sample(population=_pop,
                                  k=int(len(_pop) * test_relative)
                                 )
                   )

In [9]:
print(f'There are {len(test_ids)} traces for test')

There are 948 traces for test


In [10]:
train_ids = traces
for _test_id in test_ids:
    train_ids.remove(_test_id)

In [11]:
print(f'There are {len(train_ids)} traces for train')

There are 8710 traces for train


In [12]:
df.head()

,timestamp,trace_id,activity
0,2011-10-01 11:45:13.917000+02:00,173688,W_Completeren aanvraag
1,2011-10-01 12:17:08.924000+02:00,173688,W_Nabellen offertes
2,2011-10-08 16:32:00.886000+02:00,173688,W_Nabellen offertes
3,2011-10-10 11:33:05.791000+02:00,173688,W_Nabellen offertes
4,2011-10-13 10:37:37.026000+02:00,173688,W_Valideren aanvraag


In [13]:
test_df = df[df['trace_id'].isin(test_ids)]
train_df = df[df['trace_id'].isin(train_ids)]

In [33]:
# train_df.to_csv('datasets/train_df.csv', index=False)
# test_df.to_csv('datasets/test_df.csv', index=False)

Lets use test like a holdout dataframe

# Custom split for GridSearchCV

## Split strategy

- Split for 2 pieces that are prepared for test and train - time relations
- Random split each piece to `n_splits`, for folds


In [ ]:
class CustomCrossValidation:
    @classmethod
    def split(x: pd.DataFrame,
              y: np.ndarray = None,
              n_splits: int = None):
        """Returns to a grouped time series split generator."""
        assert 'trace_id' in x.columns.values,  ('need trace_id')
        assert 'timestamp' in x.columns.values,  ('need timestamp')
        # get traces and sort them by 
        traces = set(x['trace_id'].values)
        traces_len = traces
        
        
        
        
        
        
        
#         for group_idx in range(groups.min(), groups.max()):
#             training_group = group_idx
#             # Gets the next group right after
#             # the training as test
#             test_group = group_idx + 1
#             training_indices = np.where(
#                 groups == training_group)[0]
#             test_indices = np.where(groups == test_group)[0]
#             if len(test_indices) > 0:
#                 # Yielding to training and testing indices
#                 # for cross-validation generator
#                 yield training_indices, test_indices